In [1]:
import json, requests
import config as Config
import pandas as pd
import sys
import numpy as np
import math
from pandas.io.json import json_normalize
import re
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

conn_string = "postgresql+psycopg2://" + Config.DB_USER + ":" + Config.DB_PASS + "@localhost:5432/" + Config.DB_NAME
engine = create_engine(conn_string)

In [ ]:
park_info_url = 'http://oregonstateparks.org/data/index.cfm/parks'
park_features_url = 'http://oregonstateparks.org/data/index.cfm/features'
facilities_resp = requests.get(park_features_url)
data = json.loads(facilities_resp.text)

In [ ]:
test = json_normalize(data)

In [ ]:
test.to_csv('orparks_features.csv', index=False)

In [ ]:
parks_resp = requests.get(url)
parks_data = json.loads(parks_resp.text)
parks = json_normalize(parks_data)

In [ ]:
parks = parks.set_index('park_id')

In [ ]:
parks.to_csv('parks.csv')

In [ ]:
parkspec_features_url = 'http://oregonstateparks.org/data/index.cfm/parkFeatures'
parkspec_resp = requests.get(parkspec_features_url)
parkspec_data = json.loads(parkspec_resp.text)
df_parkspec = pd.DataFrame()
df_parkspec = json_normalize(parkspec_data)

In [ ]:
df_parkspec.head()

In [ ]:
df_parkspec.to_csv('parkspec_features.csv', index=False)

In [ ]:
test = df_parkspec.pivot('park_id', 'featureClass')['ada']

In [ ]:
test.to_csv('feature_pivot.csv')

In [ ]:
test = test.drop('', axis=1)

In [ ]:
test.to_sql('orpark_feature_ada_pivot', engine, flavor = 'postgres', if_exists='replace')

In [ ]:
parks.to_sql('orpark_parks', engine, flavor = 'postgres', if_exists='replace')

In [ ]:
orparks_baseurl = "http://www.oregonstateparks.org/index.cfm?do=parkPage.dsp_parkPage&parkId="
park_id = 105

In [ ]:
parkurl = orparks_baseurl + str(park_id)

In [ ]:
soup = BeautifulSoup(requests.get(parkurl).text, "lxml")

In [ ]:
link = soup.find_all('a', attrs={"id" : 'reserve'})

In [ ]:
link_url = str(link[0]).split("\"")[1]

In [ ]:
import HTMLParser
html_parser = HTMLParser.HTMLParser()

In [ ]:
st = html_parser.unescape(link_url)

In [ ]:
requests.get(st).text

In [ ]:
import get_orparks_data

In [ ]:
%autoreload 2

In [ ]:
get_orparks_data.get_parks()

In [ ]:
facilities_resp = requests.get(park_info_url)
data = json.loads(facilities_resp.text)
park_info = json_normalize(data)

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
get_orparks_data.merge_parks_features()

In [ ]:
orpd_demo_table = pd.read_sql('select * from orpark_demo_table', engine)

In [ ]:
orpd_demo_table = orpd_demo_table.rename(columns={'reservation_link': 'orig_reservation_link'})

In [ ]:
df = pd.DataFrame()
for index,park in orpd_demo_table.iterrows():
    if len(park['orig_reservation_link']) > 2:
        cc = park['orig_reservation_link'].split('contractCode')[1]
        reservation_link = "http://www.reserveamerica.com/facilityDetails.do?contractCode" + cc
    else:
        reservation_link = ""
    df = df.append(pd.DataFrame({ 'facilityid' : [park['facilityid']], 'reservation_link' : [reservation_link]}))


In [ ]:
merged = pd.merge(orpd_demo_table, df, how='left', on=['facilityid'])

In [ ]:
merged.head()

In [ ]:
df_avail = pd.read_csv('df_avail.csv')

In [ ]:
campsites = pd.read_sql('select * from campnear_main', engine)
facilities = campsites.drop_duplicates(subset='facilityname')
facilities = facilities.drop('cg_reservable', axis=1)

In [ ]:
merged = pd.merge(facilities, df_avail, how="left", on=['facilityname'])

In [ ]:
merged.to_sql('table_fac', engine, flavor='postgres', if_exists='replace')

In [ ]:
merged.head(10)

In [ ]:
def convert(x):
	try:
		return x.astype(int)
	except:
		return x

In [ ]:
merged.apply(convert).to_csv('table_fac.csv', index=False)

In [ ]:
merged.apply(convert).dtypes

In [ ]:
m = re.compile('.*google\.com/maps\?.*') 
print m.match("https://www.google.com/maps?q=45.59844,-122.048974")

In [ ]:
print var.group()

In [ ]:
demo_table = pd.read_sql('select * from demo_table', engine)

In [ ]:
grouped = demo_table.groupby('facilityname')

In [ ]:
sum(grouped.get_group('LOST CREEK')['campsitetype'].str.contains('YURT'))

In [ ]:
df_avail= pd.read_csv('df_avail.csv')
df_avail.to_sql('facility_availability', engine, flavor="postgres", if_exists='replace')
df_avail_samp = df_avail[['facilityname', 'last_updated', 'sites_available', 'total_sites']]
df_avail_samp.shape

In [ ]:
campsites = pd.read_sql('select * from campnear_main', engine)
facilities = campsites.drop_duplicates(subset='facilityname')

In [ ]:
merged = pd.merge(facilities, df_avail_samp, how="left", on=['facilityname'])
merged.columns
merged.to_sql('table_fac', engine, flavor='postgres', if_exists='replace')

In [ ]:
import datetime
filename = "demo_table_fac_" + "{:%m_%d}".format(datetime.date.today()) + ".csv"
command = "copy (select * from table_fac) to '/tmp/" + filename + "' delimiter ',' csv header;"
with engine.begin() as conn:
	conn.execute(command)

In [ ]:
import os
os.system("cp table_fac.csv demo_table_fac.old")
os.system("cp /tmp/" + filename + " table_fac.csv" )

In [2]:
import get_orparks_data as ORPARKS

In [3]:
allparks = pd.DataFrame()

In [4]:
orparks = pd.read_sql('select park_id from orpark_parks', engine)

In [5]:
orparks.shape

(196, 1)

In [6]:
for index,park in orparks.iterrows():
    allparks = allparks.append(ORPARKS.get_park_resp(park.park_id))

In [7]:
allparks.shape

(1351, 7)

In [8]:
allparks.columns

Index([           u'ada',   u'featureClass',   u'featureTitle',
              u'park_id',  u'park_latitude', u'park_longitude',
            u'park_name'],
      dtype='object')

In [12]:
allparks[allparks.park_id == 105]

,ada,featureClass,featureTitle,park_id,park_latitude,park_longitude,park_name
0,0,camping,Camping,105,45.59844,-122.049,Ainsworth State Park
1,0,dump,Dump Station,105,45.59844,-122.049,Ainsworth State Park
2,0,hiker,Hiker Biker,105,45.59844,-122.049,Ainsworth State Park
3,0,hiking,Hiking Trails,105,45.59844,-122.049,Ainsworth State Park
4,0,picnic,Picnicking,105,45.59844,-122.049,Ainsworth State Park
5,0,playground,Playground,105,45.59844,-122.049,Ainsworth State Park
6,0,reservable,Reservable,105,45.59844,-122.049,Ainsworth State Park
7,0,restrooms,Restrooms Flush,105,45.59844,-122.049,Ainsworth State Park
8,0,shower,Hot Shower,105,45.59844,-122.049,Ainsworth State Park
9,0,wildlife,Wildlife,105,45.59844,-122.049,Ainsworth State Park


In [50]:
test = allparks.pivot('park_id', 'featureClass')['featureTitle']

In [54]:
test.head()

featureClass,,ampitheater,beach,bike,boat,cabin,camping,dayuse,deluxecabin,deluxeyurt,...,restrooms,shower,swimming,tepee,vault,viewpoint,wildlife,windsurfing,yearround,yurt
park_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,Camping,NaN,NaN,NaN,...,Restrooms Flush,Hot Shower,NaN,NaN,NaN,NaN,Wildlife,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Restrooms Flush,NaN,NaN,NaN,NaN,NaN,Wildlife,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Boat Ramp,NaN,NaN,NaN,NaN,NaN,...,Restrooms Flush,NaN,NaN,NaN,NaN,NaN,Wildlife,NaN,NaN,NaN
4,NaN,Ampitheater,NaN,NaN,Boat Ramp,Cabin,Camping,Day-Use Fee,NaN,NaN,...,Restrooms Flush,Hot Shower,NaN,NaN,NaN,NaN,Wildlife,NaN,Open Year Round,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
test.park_id = test.index

In [40]:
test = test.drop('', axis=1)

In [42]:
test.to_sql('orpark_features', engine, flavor='postgres')

In [45]:
odf_amenities = pd.read_csv('odf_sites_edit.csv')

In [48]:
odf_tosql = odf_amenities[['facilityname', 'vault','num_sites', 'water']]

In [49]:
odf_tosql.to_sql('odf_amenities', engine, flavor='postgres')

In [56]:
orst = pd.read_csv('orst_info.csv')
fs = pd.read_csv('forestservice_info.csv')

In [57]:
orst.shape

(55, 31)

In [58]:
fs.shape

(305, 31)

In [59]:
new_main = orst.append(fs)

In [60]:
new_main.shape

(360, 31)

In [61]:
new_main.to_sql('campnear_main_new', engine, flavor='postgres')

In [62]:
new_main.columns

Index([u'cabins_available', u'cg_accessible', u'cg_conditions', u'cg_fcfs',
       u'cg_flush', u'cg_nearbeach', u'cg_nearlake', u'cg_nearriver',
       u'cg_petsallowed', u'cg_proximitytowater', u'cg_reservable',
       u'cg_season', u'cg_shower', u'cg_solitude', u'cg_status',
       u'cg_transit_avail', u'cg_vault', u'cg_water', u'drivehours',
       u'drivemiles', u'driveminutes', u'facilityid', u'facilitylatitude',
       u'facilitylongitude', u'facilityname', u'nofee', u'reservation_link',
       u'site_url', u'transithours', u'transitminutes', u'yurts_available'],
      dtype='object')